In [239]:
import requests
import re
import time
from nltk import word_tokenize
from bs4 import BeautifulSoup
from pprint import pprint
from sklearn.metrics import accuracy_score
from collections import Counter
from fake_useragent import UserAgent
from pymorphy2 import MorphAnalyzer
ua = UserAgent(verify_ssl=False)
session = requests.session()
morph = MorphAnalyzer()

апд: у 30-ти хороших отзывов оче мало частотных слов, возьму больше. Люди не любят хвалить    

Всего мне нужно по **35 хороших и плохих отзывов** (по 30 для тренировки и 5 для проверки). На одной странице сайта отображается **30** отзывов, так что мне хватит всего двух страниц.  
**Плохие отзывы:**  
https://www.otzyvru.com/pochta-rossii?rating=1  
**Хорошие отзывы:**  
https://www.otzyvru.com/pochta-rossii?rating=5

In [240]:
a = 'https://www.otzyvru.com/pochta-rossii?rating=5'
b = 'https://www.otzyvru.com/pochta-rossii?rating=5&page=2'
c = 'https://www.otzyvru.com/pochta-rossii?rating=1'
d = 'https://www.otzyvru.com/pochta-rossii?rating=1&page=2'
good_links = [a, b]
bad_links = [c, d]

In [241]:
# собирает список ссылок с одной страницы
def parse_page(link, number):
    responce = session.get(link)
    html = responce.text
    soup = BeautifulSoup(html, 'html.parser')
    links = []
    for h in soup.select('h2'):
        if len(links) < number:
            l = h.find('a').attrs['href']
            links.append(l)
    return links

In [242]:
# собирает список ссылок с обеих страниц
def parse_two_pages(link):
    first_p = parse_page(link, 30)
    second_p = parse_page(link+'&page=2', 20)
    full_links = first_p + second_p   
    return full_links

In [243]:
# токенизация и начальная форма
def process(text):
    lem_text = []
    for word in word_tokenize(text.lower()):
        if word.isalpha():
            result = morph.parse(word)[0]
            lemma = result.normal_form
            lem_text.append(lemma)
    return lem_text

In [250]:
# собирает текст отзывов в виде общего списка
def grab_reviews(links):
    revs = []
    for link in links:
        responce = session.get(link)
        html = responce.text
        soup = BeautifulSoup(html, 'html.parser')
        review = soup.find('span', {'class': 'comment description'}).text
        lem_rev = process(review)
        revs = revs.append(lem_rev)
    return revs

In [245]:
good = 'https://www.otzyvru.com/pochta-rossii?rating=5'
bad = 'https://www.otzyvru.com/pochta-rossii?rating=1'
good_links = parse_two_pages(good)
bad_links = parse_two_pages(bad)

In [246]:
good_list = grab_reviews(good_links)
bad_list = grab_reviews(bad_links)

**Распределение слов**

In [125]:
good_only = [g for g in good_list if g not in bad_list]
bad_only = [b for b in bad_list if b not in good_list]

In [133]:
def get_freqlist(word_list):
    freqlist = Counter()
    for word in word_list:
        freqlist[word] += 1
    return dict(freqlist.most_common(100))

In [159]:
freqlists = {}
good_freq = get_freqlist(good_only)
bad_freq = get_freqlist(bad_only)
freqlists['Это положительный отзыв'] = good_freq
freqlists['Это отрицательный отзыв'] = bad_freq

In [270]:
test_text = 'Спасибо за вашу ерунду. много очередей'

In [271]:
def analyze(freqlists, text):
  #создадим счётчик, который будет для каждого языка хранить информацию о том, сколько слов из поданного текста являются словами этого языка 
  #или нет 
    counts = Counter()
    for sentiment, freqlist in freqlists.items():
        freqlist = Counter(freqlist)
        for word in text:
            counts[sentiment] += int(freqlist[word] > 0)
    result = counts.most_common()
    nice_output = 'Вердикт: '+str(result[0][0])+' с шансами '+str(result[0][1])+' к '+str(result[1][1])
    return result, nice_output

In [272]:
print(analyze(freqlists, process(test_text))[1])

Вердикт: Это положительный отзыв с шансами 2 к 0


In [220]:
# собрать 10 тестовых текстов

In [256]:
# отличается от grab_reviews() тем, что делает подсписки, а не всё в кучу
def grab_test(links):
    revs = []
    for link in links:
        responce = session.get(link)
        html = responce.text
        soup = BeautifulSoup(html, 'html.parser')
        review = soup.find('span', {'class': 'comment description'}).text
        lem_rev = process(review)
        revs.append(lem_rev)
    return revs

In [262]:
def test_detection(freqlists):
    test_list = []
    gold = ['Это положительный отзыв']*5+['Это отрицательный отзыв']*5
    results = []
    test_links = ['https://www.otzyvru.com/pochta-rossii?rating=5&page=3','https://www.otzyvru.com/pochta-rossii?rating=1&page=3']
    for link in test_links:
        test_list += grab_test(parse_page(link, 5))
    for text in test_list:
        predicted_sentiment = analyze(freqlists, text)[0][0][0]
        results.append(predicted_sentiment)
    print("Accuracy: %.4f" % accuracy_score(results, gold))

In [263]:
test_detection(freqlists)

Accuracy: 0.8000


**Идеи улучшения кода:**